In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
sc.stop()

In [63]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import sklearn
import random 

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit,sum
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import pandas as pd
import numpy as np
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [64]:
# 2019년 10월 데이터(4분 소요)
df_Oct = spark.read.csv("/content/drive/MyDrive/2022 빅데이터처리및응용 팀프로젝트/2019-Oct.csv", inferSchema=True, header=True)
df_Oct.show()

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    null|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

In [65]:
# 2019년 11월 데이터(6분 소요)
df_Nov = spark.read.csv("/content/drive/MyDrive/2022 빅데이터처리및응용 팀프로젝트/2019-Nov.csv", inferSchema=True, header=True)
df_Nov.show()

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                null|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.s

In [66]:
# 데이터 통합 및 결측치 있는 행과 불필요한 열 제거
df_2019=df_Oct.union(df_Nov)
df_2019=df_2019.dropna()
df_2019=df_2019.drop('user_session', 'event_time')
df_2019.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)



In [67]:
df_2019.show(5)

+----------+----------+-------------------+--------------------+------+-------+---------+
|event_type|product_id|        category_id|       category_code| brand|  price|  user_id|
+----------+----------+-------------------+--------------------+------+-------+---------+
|      view|   3900821|2053013552326770905|appliances.enviro...|  aqua|   33.2|554748717|
|      view|   1307067|2053013558920217191|  computers.notebook|lenovo| 251.74|550050854|
|      view|   1004237|2053013555631882655|electronics.smart...| apple|1081.98|535871217|
|      view|   1480613|2053013561092866779|   computers.desktop|pulser| 908.62|512742880|
|      view|  28719074|2053013565480109009|  apparel.shoes.keds| baden| 102.71|520571932|
+----------+----------+-------------------+--------------------+------+-------+---------+
only showing top 5 rows



In [68]:
# 약 30분 소요

# 이벤트 타입 중요도에 따라 제품에 대한 클릭 가중치 점수 임의로 지정(view=0.1,cart=0.2,purchase=0.3)
df_2019=df_2019.withColumn("product_score",when(df_2019.event_type == 'view',0.1)
    .when(df_2019.event_type == 'cart',0.2)
    .when(df_2019.event_type == 'purchase',0.3))

# user별 각 제품에 대한 가중치 총점을 구한다
df_sum=df_2019.groupBy("user_id","product_id").agg(sum("product_score").alias("total_score"))

# user_id와 product_id가 같은 요소들 중에서 중복 제거 및 product_score,event_type,category_id 열 제거
df_2019=df_2019.dropDuplicates(["user_id","product_id"])
df_2019=df_2019.drop("product_score","event_type","category_id")

# user_id,product_id 열로 join하여 total_score열 추가
df_joined=df_2019.join(df_sum,["user_id","product_id"])

df_joined.show()

+---------+----------+--------------------+----------+-------+-----------+
|  user_id|product_id|       category_code|     brand|  price|total_score|
+---------+----------+--------------------+----------+-------+-----------+
| 31198833|   1005065|electronics.smart...|    xiaomi| 218.54|        0.1|
| 88309646|   1003906|electronics.smart...|    huawei| 123.07|        0.1|
| 96041329|   5100850|  electronics.clocks|    huawei| 154.16|        0.1|
|116566414|   2701022|appliances.kitche...|  leadbros|  198.2|        0.1|
|116566414|   2701889|appliances.kitche...|      beko| 360.08|        0.1|
|116566414|   2702522|appliances.kitche...|  dauscher| 185.31|        0.2|
|126473256|  17800342|   computers.desktop|      zeta|   66.9|        0.1|
|128968633|   1802004|electronics.video.tv|     yasin| 253.49|        0.1|
|149382035|   3900652|appliances.enviro...|   ariston| 118.38|        0.1|
|199915639|   4700330|auto.accessories....|      ibox| 131.28|        0.1|
|213763705|   2500577|app

In [69]:
#@title 기본 제목 텍스트
#사람들이 구매한 브랜드 종류 살펴보기()
from pyspark.sql import SparkSession
df_joined.select("brand").distinct().show()

+--------+
|   brand|
+--------+
| tuffoni|
| edifier|
|    tmnt|
|  vortex|
|  marley|
|   goo.n|
|   fagor|
| keenway|
|eurogold|
|   dagee|
|   sigma|
| ariston|
|   stagg|
| versace|
|decoroom|
| kettler|
| camshel|
|  mackie|
|  rieker|
|  wagner|
+--------+
only showing top 20 rows



In [71]:
#유저가 구매한 물건 확인하기
df2=df_joined.where(df_joined.user_id=='219406386')
df2.show()

+---------+----------+--------------------+----------+------+-----------+
|  user_id|product_id|       category_code|     brand| price|total_score|
+---------+----------+--------------------+----------+------+-----------+
|219406386|  13200475|furniture.bedroom...|permanence|864.86|        0.1|
|219406386|  13200221|furniture.bedroom...|     vegas|810.32|        0.1|
+---------+----------+--------------------+----------+------+-----------+



In [ ]:
# ALS recommender algorithm(1시간 반정도 소요)
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# user_id, product_id, total_score 3개의 열만 추출하여 Dataframe 생성
df_data=df_joined.select("user_id","product_id","total_score")

train, test = df_data.randomSplit([0.8, 0.2])

rec = ALS(maxIter=10,
         regParam=0.01,
         userCol='user_id',
         itemCol='product_id',
         ratingCol='total_score',
         nonnegative=True,
         coldStartStrategy='drop')
# ALS모델 학습
rec_model = rec.fit(train)

# transform을 이용해 예측
prediction = rec_model.transform(test)

# 예측 성능 평가(rmse)
evaluator = RegressionEvaluator(labelCol='total_score',
                               predictionCol='prediction',
                               metricName='rmse')
# evaluate 메소드에 예측값 담겨있는 dataframe 넣어주기
rmse = evaluator.evaluate(prediction)

print("RMSE:", rmse)

RMSE: 0.4450871956614777


In [ ]:
# 30분 소요

# product_id Dataframe 생성
products = df_joined.select("product_id").distinct()
# df_joined Dataframe에서 total_score,user_id 열을 뺀 나머지 정보
products_info=df_joined.drop("total_score","user_id")
products_info=products_info.dropDuplicates(["product_id"])

# 특정 user_id가 좋아할 만한 n개의 제품 추천해주는 함수
def Recommend_products(user_id,n):

    # products을을 'p'라는 Dataframe으로 바꿔준다
    p = products.alias('p')

    # 특정 user_id가 본 제품들만 담은 새로운 Dataframe 생성
    watched_products = df_joined.filter(df_joined['user_id'] == user_id)\
                            .select('product_id')

    # watched_products을 'w'라는 Dataframe으로 바꿔준다
    w = watched_products.alias('w')

    # products를 기준으로 watched_products를 조인시켜서 user_id가 보지 못한 제품들 파악 가능
    total_products = p.join(w, p['product_id'] == w['product_id'],
                         how='left')

    # b의 product_id 값이 결측치를 갖고 있는 행의 a.product_id를 뽑아냄으로써 user_id가 아직 못본 제품들 추출
    unwatched_products = total_products\
                       .where(col('w.product_id').isNull())\
                       .select('p.product_id').distinct()

    # unwatched_products Dataframe에 특정 user_id값을 새로운 열로 추가해주기
    unwatched_products = unwatched_products.withColumn('user_id',lit(int(user_id)))

    # 위에서 만든 ALS 모델을 사용하여 가중치 총점 예측 후 n개 만큼 보여준다
    recommender = rec_model.transform(unwatched_products)\
                           .orderBy('prediction', ascending=False)\
                           .limit(n)

    # 제품 종류, 브랜드, 가격 정보도 join을 통해 결과에 같이 보여준다
    recommender=recommender.join(products_info,["product_id"])

    # 가중치 총점 기준으로 내림 차순 재정렬
    recommender=recommender.orderBy('prediction', ascending=False)

    return recommender.show(n, truncate=False)

# user_id가 219406386인 유저가 볼만한 제품 상위 5개 추천해주기
Recommend_products(219406386,5)

+----------+---------+----------+-----------------------------------+-------+------+
|product_id|user_id  |prediction|category_code                      |brand  |price |
+----------+---------+----------+-----------------------------------+-------+------+
|5100789   |219406386|0.68104655|electronics.clocks                 |garmin |236.5 |
|1701356   |219406386|0.5485222 |computers.peripherals.monitor      |asus   |549.8 |
|44800012  |219406386|0.4855878 |sport.tennis                       |babolat|164.48|
|100009895 |219406386|0.47270906|computers.components.power_supply  |corsair|137.47|
|3901170   |219406386|0.46167257|appliances.environment.water_heater|ariston|179.63|
+----------+---------+----------+-----------------------------------+-------+------+

